In [ ]:
from kafka import KafkaProducer
from pyspark.sql import SparkSession
import time

def producer():
    # Spark session & context
    spark = (SparkSession
             .builder
             .master('local')
             .appName('kafka-streaming')
             # Add kafka package
             .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5")
             .getOrCreate())
    sc = spark.sparkContext
    
    # Khởi tạo Kafka producer
    bootstrap_servers = 'kafka:9092'  # Thay đổi tùy theo cấu hình Kafka của bạn
    topic = 'trading'  # Tên topic Kafka để gửi dữ liệu
    producer = KafkaProducer(bootstrap_servers=bootstrap_servers)
    
    # Read the message from the kafka stream
    df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka:9092") \
        .option("subscribe", topic) \
        .load()
    
    df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
    
    #Create a small temporary view for SparkSQL
    df.createOrReplaceTempView("message")
    
    # Write out the message to the console of the environment
    res = spark.sql("SELECT * from message")
    res.writeStream.format("console") \
                .outputMode("append") \
                .start() 
    
# Hàm gửi dữ liệu đến Kafka
def send_data_to_kafka(data):
    producer.send(topic, value=data.encode('utf-8'))
    

In [ ]:
# Write the message back into Kafka in another topic that you are going to listen to with a local consumer
ds = df \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", topic) \
    .option("checkpointLocation", "/tmp") \
    .start()

# Tạo và gửi dữ liệu mô phỏng đến Kafka
i=0
while True:
    i=i+1
    data = f"Sample data{i}"  # Thay đổi dữ liệu mô phỏng tại đây
    print(data)
    send_data_to_kafka(data)
    # Thời gian chờ giữa các lần gửi dữ liệu
    # Chú ý rằng bạn có thể điều chỉnh thời gian chờ này để phù hợp với tốc độ xử lý của Spark Streaming
    time.sleep(1)


In [5]:
# import findspark
# findspark.init()

import pyspark
import random

sc = pyspark.SparkContext(appName="Pi")
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

sc.stop()

3.14143116
